Countries Financing

FOR GCF use data from website

In [19]:
import pandas as pd

data = pd.read_excel("C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\country_financing\\GCF_ODL-Export-projects-1713963203661.xlsx")

# count countries with 1 if 1
def count_countries(countries):
    country_list = [c.strip() for c in countries.split(',')]
    return max(1, len(country_list)) 

data['Country Count'] = data['Countries'].apply(count_countries)

def calculate_financing_per_country(row):
    return row['FA Financing'] / row['Country Count'] 

data['FA Financing per Country'] = data.apply(calculate_financing_per_country, axis=1)


expanded_df = data.assign(Countries=data['Countries'].str.split(',')).explode('Countries')
expanded_df['Countries'] = expanded_df['Countries'].str.strip()
expanded_grouped = expanded_df.groupby('Countries').size().reset_index(name='Countries No of Projects')
expanded_df = expanded_df.merge(expanded_grouped, on='Countries', how='left')
expanded_df.to_excel('C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\country_financing\\gcf_expanded.xlsx', index=False)
data1_grouped = expanded_df.groupby('Countries').agg({'FA Financing per Country': 'sum'}).reset_index()
file_path2 = "C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\country_financing\\GCF_ODL-Export-countries-1713963078444.xlsx"
data2 = pd.read_excel(file_path2)
data2_renamed = data2.rename(columns={'Country Name': 'Countries', 'FA Financing $': 'FA Financing from Data2'})
comparison_df = pd.merge(data1_grouped, data2_renamed[['Countries', 'FA Financing from Data2']], on='Countries', how='outer')
comparison_df.fillna(0, inplace=True)
comparison_df['Financing Difference'] = comparison_df['FA Financing per Country'] - comparison_df['FA Financing from Data2']
comparison_df['Percentage Difference'] = (comparison_df['Financing Difference'] / comparison_df['FA Financing from Data2'].replace(0, pd.NA)) * 100
comparison_df.sort_values(by='Percentage Difference', ascending=False, inplace=True)


comparison_df.to_excel('C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\country_financing\\gcf_comparison_financing.xlsx', index=False)



# Merge with data gef countries financing 
expanded_df = expanded_df.merge(data2_renamed[['Countries', 'FA Financing from Data2']], on='Countries', how='left')
expanded_df['Avg Pro Finance'] = expanded_df['FA Financing from Data2'] / expanded_df['Countries No of Projects']
expanded_df['Avg Pro Finance'].fillna(0, inplace=True)

# Save the final DataFrame
expanded_df.to_excel('C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\country_financing\\GCF_expanded_with_avg_finance.xlsx', index=False)




C:\Users\david\AppData\Local\Temp\ipykernel_12300\2402963311.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  expanded_df['Avg Pro Finance'].fillna(0, inplace=True)


,Ref #,Modality,Project Name,Entity,Countries,BM,Sector,Theme,Project Size,ESS Category,FA Financing,Country Count,FA Financing per Country,Countries No of Projects,FA Financing from Data2,Avg Pro Finance
0,FP001,PAP,Building the Resilience of Wetlands in the Pro...,Profonanpe,Peru,B.11,Public,Cross-cutting,Micro,Category C,6240000.0,1,6240000.0,8,1.157177e+08,1.446471e+07
1,FP002,PAP,Scaling up the use of Modernized Climate infor...,UNDP,Malawi,B.11,Public,Adaptation,Small,Category C,12295000.0,1,12295000.0,5,4.438516e+07,8.877032e+06
2,FP003,PAP,Increasing the resilience of ecosystems and co...,CSE,Senegal,B.11,Public,Adaptation,Micro,Category C,7610000.0,1,7610000.0,14,1.938931e+08,1.384950e+07
3,FP004,PAP,Climate Resilient Infrastructure Mainstreaming...,KfW,Bangladesh,B.11,Public,Adaptation,Medium,Category B,40000000.0,1,40000000.0,9,4.411650e+08,4.901834e+07
4,FP005,PAP,KawiSafi Ventures Fund,Acumen,Kenya,B.11,Private,Cross-cutting,Micro,Intermediation 3,25000000.0,2,12500000.0,19,2.810083e+08,1.478991e+07


for gef (use file from the website)

In [22]:
import pandas as pd

data = pd.read_csv("C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\country_financing\\gef_projects.csv")


data['GEF Grant'] = data['GEF Grant'].str.replace(',', '').astype(float)

# Function to count countries
def count_countries(countries):
    country_list = [c.strip() for c in countries.split(',')]
    if len(country_list) == 1:
        return 1 
    return len(country_list)

data['Country Count'] = data['Countries'].apply(count_countries)

# grant per country
def calculate_grant_per_country(row):
    if row['Country Count'] > 1:  
        return row['GEF Grant'] / row['Country Count']
    return row['GEF Grant']
data['GEF Grant per Country'] = data.apply(calculate_grant_per_country, axis=1)

# Explode df
expanded_data = data.assign(Countries=data['Countries'].str.split(',')).explode('Countries')
expanded_data['Countries'] = expanded_data['Countries'].str.strip()  # Strip whitespace from country names

# Replace 'GEF Grant' with 'GEF Grant per Country' when there are multiple countries
expanded_data['GEF Grant'] = expanded_data.apply(
    lambda row: row['GEF Grant per Country'] if row['Country Count'] > 1 else row['GEF Grant'], 
    axis=1
)


expanded_data['Avg Pro Finance'] = expanded_data['GEF Grant'] / expanded_data['Country Count']

expanded_data.to_csv('C:\\Users\\david\\My Drive\\data\\analysis_git_data\\cgiar\\country_financing\\GEF_expanded_with_avg_finance.csv', index=False)


FOR AF